In [85]:
import os
from pathlib import Path
import pandas as pd
from src import utils
import math
import re
from tqdm import tqdm

In [86]:
# Project path
project_dir = str(Path().resolve().parents[1])
# Load enviromental variables
env_var = utils.load_env_variables(project_dir)

In [88]:

dataset = "Brazil_Election_2018"
validation = "CrossValidation"
ml_methods = ["KNN"]
fs_method = "CFS"
index_col = "INDEX"

In [89]:
data_path = os.path.join(env_var["root_path"], dataset, "Original", "data.csv")
fold_path = os.path.join(env_var["root_path"], dataset, "Original", "folds", validation)
features_path = os.path.join(env_var["root_path"], dataset, "Original", "results", validation, "features_selected", fs_method)

In [90]:
data = pd.read_csv(data_path, index_col=index_col)
data[["[GEO]_LATITUDE", "[GEO]_LONGITUDE"]]=(data[["[GEO]_LATITUDE", "[GEO]_LONGITUDE"]]-data[["[GEO]_LATITUDE", "[GEO]_LONGITUDE"]].min())/(data[["[GEO]_LATITUDE", "[GEO]_LONGITUDE"]].max()-data[["[GEO]_LATITUDE", "[GEO]_LONGITUDE"]].min())
data

,INDEX_FOLDS,TARGET,[GEO]_LATITUDE,[GEO]_LONGITUDE,[CENSUS]_DOMICILIO01_V002,[CENSUS]_DOMICILIO01_V003,[CENSUS]_DOMICILIO01_V004,[CENSUS]_DOMICILIO01_V005,[CENSUS]_DOMICILIO01_V006,[CENSUS]_DOMICILIO01_V007,...,[CENSUS]_RESPONSAVELRENDA_V123,[CENSUS]_RESPONSAVELRENDA_V124,[CENSUS]_RESPONSAVELRENDA_V125,[CENSUS]_RESPONSAVELRENDA_V126,[CENSUS]_RESPONSAVELRENDA_V127,[CENSUS]_RESPONSAVELRENDA_V128,[CENSUS]_RESPONSAVELRENDA_V129,[CENSUS]_RESPONSAVELRENDA_V130,[CENSUS]_RESPONSAVELRENDA_V131,[CENSUS]_RESPONSAVELRENDA_V132
INDEX,,,,,,,,,,,,,,,,,,,,,
1200013,12,77.124085,0.618703,0.158198,0.995969,0.988771,0.001440,0.002591,0.765621,0.005471,...,0.032575,0.011371,0.008421,0.004347,0.008772,0.002226,0.039279,0.069073,0.051785,0.149553
1200054,12,56.393522,0.596654,0.083705,0.983387,0.957827,0.000000,0.001278,0.757827,0.013419,...,0.063155,0.027715,0.033041,0.013889,0.004027,0.000000,0.000000,0.089870,0.074352,0.193800
1200104,12,70.723390,0.597497,0.103165,0.992801,0.956136,0.001005,0.028461,0.757743,0.030470,...,0.039175,0.017606,0.024446,0.022189,0.006045,0.004436,0.002080,0.098378,0.079099,0.172588
1200138,12,75.431521,0.627042,0.128374,0.994027,0.975256,0.000427,0.005119,0.689846,0.020904,...,0.028636,0.010683,0.013681,0.012133,0.009122,0.003766,0.000000,0.060733,0.051150,0.116068
1200179,12,75.881551,0.604620,0.136607,0.996612,0.976705,0.000000,0.013977,0.675985,0.055909,...,0.039760,0.013020,0.007314,0.021257,0.009113,0.000000,0.000000,0.074099,0.063465,0.139319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1721208,17,30.394314,0.714605,0.631512,0.994384,0.947850,0.000481,0.000321,0.762356,0.005456,...,0.035768,0.015773,0.027886,0.024893,0.003475,0.003037,0.004049,0.119128,0.103794,0.182833
1721257,17,39.984165,0.644668,0.613913,1.000000,1.000000,0.000000,0.000000,0.700837,0.006276,...,0.045282,0.009829,0.015786,0.016486,0.000000,0.000000,0.000000,0.070521,0.059085,0.116922
1721307,17,27.090032,0.659837,0.615096,1.000000,1.000000,0.000000,0.000000,0.823529,0.000000,...,0.054498,0.037234,0.018434,0.021546,0.000000,0.000000,0.000000,0.117310,0.095374,0.222081


In [91]:
from genericpath import isfile
from ntpath import join

results_method = {}
folds = [f for f in os.listdir(fold_path) if not os.path.isfile(os.path.join(fold_path, f))]
for ml_method in tqdm(ml_methods):
    features_abblation = []
    mse_abblation = []
    features_all = []
    for ablation_perc in tqdm([0]):
        mse_by_fold = []
        len_features = []
        for fold in folds:
            splited_data = utils.load_json(os.path.join(fold_path,fold,"split_data.json"))
            features = utils.load_json(os.path.join(features_path,f"{fold}.json"))
            if ablation_perc > -1:
                init_pointer = int(ablation_perc*len(features["selected_features"]))
                features["selected_features"] = features["selected_features"][: len(features["selected_features"]) - init_pointer]
                features["selected_features"] = features["selected_features"] + ["[GEO]_LATITUDE", "[GEO]_LONGITUDE"]
            X_train = data.loc[splited_data["train"],features["selected_features"]].copy()
            X_test = data.loc[splited_data["test"],features["selected_features"]].copy()
            y_train = data.loc[splited_data["train"],"TARGET"].copy()
            y_test = data.loc[splited_data["test"],"TARGET"].copy()

            clean_cols = [re.sub(r"\W+", "", col) for col in X_train.columns]
            X_train.columns = clean_cols
            X_test.columns = clean_cols

            model = get_model(ml_method, len(X_train.columns), {})
            model.fit(X_train,y_train)
            pred = model.predict(X_test)
            mse_by_fold.append(((y_test - pred)**2).mean())
            len_features.append(len(X_test.columns))
            features_all = features_all + features["selected_features"]
        features_abblation.append(sum(len_features)/len(len_features))
        mse_abblation.append(sum(mse_by_fold)/len(mse_by_fold))
    results_method[ml_method] = mse_abblation

100%|██████████| 1/1 [00:00<00:00,  1.72it/s]


In [92]:
features_all_folds = list(set(features_all))
features_all_folds.append("TARGET")
data_state = data[data["INDEX_FOLDS"]==23]
corr = data_state[features_all_folds].corr()
#corr.style.background_gradient(cmap='coolwarm')

In [93]:
pd.set_option('display.max_rows', 500)
corr.loc["TARGET"]

[CENSUS]_PESSOA03_V212            0.092604
[CENSUS]_PESSOA04_V072           -0.048031
[CENSUS]_PESSOA13_V128           -0.251086
[CENSUS]_PESSOARENDA_V044         0.655445
[CENSUS]_PESSOA03_V006           -0.030209
[CENSUS]_ENTORNO05_V939                NaN
[CENSUS]_PESSOA06_V023            0.182804
[CENSUS]_PESSOA09_V134            0.134433
[CENSUS]_PESSOA11_V128           -0.237855
[CENSUS]_PESSOA03_V237           -0.177660
[CENSUS]_PESSOA09_V149            0.041880
[CENSUS]_RESPONSAVEL02_V091      -0.066920
[CENSUS]_PESSOA11_V132           -0.038104
[CENSUS]_PESSOA03_V047            0.118420
[CENSUS]_PESSOA02_V118            0.492121
[CENSUS]_PESSOA11_V125           -0.357730
[CENSUS]_PESSOA08_V019            0.311145
[CENSUS]_RESPONSAVEL02_V030       0.251165
[CENSUS]_PESSOA08_V119           -0.319041
[CENSUS]_RESPONSAVEL02_V216      -0.157212
[CENSUS]_ENTORNO05_V1026               NaN
[CENSUS]_PESSOA07_V023           -0.217567
[CENSUS]_RESPONSAVEL02_V205       0.529071
[CENSUS]_RE

In [94]:
results = pd.DataFrame(results_method)
#results.to_csv("results.csv", index=False)
#results

In [95]:
(results.loc[1] - results.loc[0]).plot()

KeyError: 1